In [1]:
pip install plotly pandas dash

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots
import plotly.graph_objects as go  # Added import for go

In [11]:
mef_m_clean = pd.read_csv('/Users/michaelgeiser/Desktop/MScQF/Lectures/Fall 2023/DTF/data/clean_data/mef_m_clean.csv')
mkt_m_clean = pd.read_csv('/Users/michaelgeiser/Desktop/MScQF/Lectures/Fall 2023/DTF/data/clean_data/mkt_m_clean.csv')

In [12]:
def normalize_columns(df):
    """Normalize all columns in a DataFrame except 'date'."""
    df_normalized = df.copy()
    for column in df.columns:
        if column != 'date':
            df_normalized[column] = (df[column] - df[column].mean()) / df[column].std()
    return df_normalized

mef_m_clean = normalize_columns(mef_m_clean)
mkt_m_clean = normalize_columns(mkt_m_clean)

In [23]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("MEF Data vs GSPC"),
    
    # Checklist for selecting columns in mef_m_clean
    html.Label("Select Macro Economic Factors:"),
    dcc.Checklist(
        id='column-dropdown',
        options=[{'label': col, 'value': col} for col in mef_m_clean.columns if col != 'date'],
        value=mef_m_clean.columns[:5],  # Set default values
        inline=True,  # Display options horizontally
        style={'columnCount': 1}  # Arrange options into one column
    ),
    
    # Dropdown for selecting GSPCprem or lead_GSPCprem_1 in mkt_m_clean
    html.Label("Select GSPCprem or lead_GSPCprem_1:"),
    dcc.Dropdown(
        id='mkt-column-selector',
        options=[
            {'label': 'GSPCprem', 'value': 'GSPCprem'},
            {'label': 'lead_GSPCprem_1', 'value': 'lead_GSPCprem_1'}
        ],
        value='GSPCprem',  # Set default value
        multi=False
    ),
    
    # DatePickerRange for selecting start and end dates
    html.Label("Select Date Range:"),
    dcc.DatePickerRange(
        id='date-range-selector',
        start_date=mef_m_clean['date'].min(),
        end_date=mef_m_clean['date'].max(),
        display_format='YYYY-MM-DD'
    ),
    
    # Plotly figure
    dcc.Graph(id='line-plot')
])

# Callback to update the plot based on user input
@app.callback(
    Output('line-plot', 'figure'),
    [Input('column-dropdown', 'value'),
     Input('date-range-selector', 'start_date'),
     Input('date-range-selector', 'end_date')]
)
def update_plot(selected_columns, start_date, end_date):
    # Filter data based on date range
    filtered_mef_m_clean = mef_m_clean[(mef_m_clean['date'] >= start_date) & (mef_m_clean['date'] <= end_date)]
    filtered_mkt_m_clean = mkt_m_clean[(mkt_m_clean['date'] >= start_date) & (mkt_m_clean['date'] <= end_date)]

    # Create subplots with shared x-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add traces for mef_m_clean (excluding 'date')
    for i, column in enumerate(selected_columns):
        if column != 'date':
            linestyle = ['solid', 'dash', 'dot', 'dashdot'][i % 4]  # Use different line styles
            fig.add_trace(
                go.Scatter(x=filtered_mef_m_clean['date'], y=filtered_mef_m_clean[column],
                           name=f'MEF: {column}', line=dict(color='blue', dash=linestyle)),
                secondary_y=False,
            )

    # Add trace for GSPCprem or lead_GSPCprem_1 in mkt_m_clean
    mkt_column = 'GSPCprem'  # Change to 'lead_GSPCprem_1' if needed
    fig.add_trace(
        go.Scatter(x=filtered_mkt_m_clean['date'], y=filtered_mkt_m_clean[mkt_column],
                   name=f'MKT: {mkt_column}', line=dict(color='red', dash='solid')),
        secondary_y=True,
    )

    # Update layout
    fig.update_layout(xaxis_title="Date")

    # Update y-axis labels and set manual ranges
    fig.update_yaxes(title_text="MEF Data (Normalized)", secondary_y=False, range=[-6, 6])  # Set your desired range
    fig.update_yaxes(title_text="GSPC Data", secondary_y=True, range=[-6, 6])  # Set your desired range

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)